# Plot Summary

Create publication-ready visualizations of PyPSA optimization results including energy balances, installed capacities, and system costs.

**How to use:**
1. Run `make_stats_dict.py` to extract statistics from network files
2. Update `plot_summary.yaml` with your experiment configuration (countries, years, scenarios)
3. Execute cells below to generate all plots automatically

**Configuration:** Edit `plot_summary.yaml` to switch between experiments, filter scenarios, or adjust plot styling without changing code.

**Environment:** Install `envs/pypsa-earth-vis.yaml`

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import pandas as pd
idx_slice = pd.IndexSlice
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import plotly.express as px

# import sys
# sys.path.append('../scripts')
from plot_helpers import (
    chdir_to_root_dir,
    read_stats_dict,
    prepare_dataframe,
    rename_electricity, 
    rename_h2,
    rename_gas,
    rename_oil,
    rename_co2,
    rename_costs,
    colors,
    save_plotly_fig,
    get_scen_col_function,
    apply_standard_styling,
    plot_energy_balance,
    plot_capacity,
    load_plot_config
)

chdir_to_root_dir()

In [2]:
# Load configuration from YAML file
yaml_config = load_plot_config()

# Extract configuration
RUN_NAME_PREFIX = yaml_config['run']['name_prefix']
SUMMARY_VERSION = yaml_config['run']['summary_version']
SDIR = Path.cwd() / "results" / f"{RUN_NAME_PREFIX}_summary_{SUMMARY_VERSION}"
SDIR.mkdir(exist_ok=True, parents=True)

COUNTRIES = yaml_config['data']['countries']
YEARS = yaml_config['data']['years']
INDEX_LEVELS_TO_DROP = yaml_config['data']['index_levels_to_drop']

# Build index group
IDX_GROUP = idx_slice[[RUN_NAME_PREFIX], :, COUNTRIES, YEARS]
IDX_GROUP_NAME = "".join(COUNTRIES) + "_MAIN"

print(f"Index Group Name: {IDX_GROUP_NAME}")
print(f"Index Group: {IDX_GROUP}")

# Scenario filtering
SCEN_FILTER = yaml_config['data']['scen_filter']
scen_order = yaml_config['data']['scen_order']

# Get scenario column function
scen_col_func_name = yaml_config['data']['scen_col_function']
set_scen_col = get_scen_col_function(scen_col_func_name)
print(f"Using scenario column function: {scen_col_func_name}")

# Plot dimensions
width = yaml_config['plot']['width']
heights = yaml_config['plot']['heights']

# Default figure kwargs
fig_kwargs = yaml_config['plot']['default_kwargs'].copy()
fig_kwargs['width'] = width
fig_kwargs['height'] = heights['medium']

print(f"\nConfiguration loaded successfully!")
print(f"Countries: {COUNTRIES}")
print(f"Years: {YEARS}")
print(f"Scenario filter: {SCEN_FILTER if SCEN_FILTER else 'None (show all)'}")

Index Group Name: CD_MAIN
Index Group: (['scenarios_H2G'], slice(None, None, None), ['CD'], [2035])
Using scenario column function: H2G

Configuration loaded successfully!
Countries: ['CD']
Years: [2035]
Scenario filter: None (show all)


In [3]:
# Load data dictionaries using configuration
balance_keys = yaml_config['carriers']['balance']
capacity_keys = yaml_config['carriers']['capacity']
cost_keys = yaml_config['carriers']['costs']

balance_dict = read_stats_dict("balance_dict", SDIR, keys=balance_keys)
optimal_capacity_dict = read_stats_dict("optimal_capacity_dict", SDIR, keys=capacity_keys)
costs_dict = read_stats_dict("costs_dict", SDIR, keys=cost_keys)

print(f"Loaded balance data for: {balance_keys}")
print(f"Loaded capacity data for: {capacity_keys}")
print(f"Loaded cost data for: {cost_keys}")

Imported AC from /mnt/data/gsa/pypsa-earth-lukas/results/scenarios_H2G_summary_v2/balance_dict_AC.csv
Imported H2 from /mnt/data/gsa/pypsa-earth-lukas/results/scenarios_H2G_summary_v2/balance_dict_H2.csv
Imported oil from /mnt/data/gsa/pypsa-earth-lukas/results/scenarios_H2G_summary_v2/balance_dict_oil.csv
Imported gas from /mnt/data/gsa/pypsa-earth-lukas/results/scenarios_H2G_summary_v2/balance_dict_gas.csv
Imported co2 stored from /mnt/data/gsa/pypsa-earth-lukas/results/scenarios_H2G_summary_v2/balance_dict_co2 stored.csv
Imported co2 from /mnt/data/gsa/pypsa-earth-lukas/results/scenarios_H2G_summary_v2/balance_dict_co2.csv
Imported AC from /mnt/data/gsa/pypsa-earth-lukas/results/scenarios_H2G_summary_v2/optimal_capacity_dict_AC.csv
Imported H2 from /mnt/data/gsa/pypsa-earth-lukas/results/scenarios_H2G_summary_v2/optimal_capacity_dict_H2.csv
Imported capex from /mnt/data/gsa/pypsa-earth-lukas/results/scenarios_H2G_summary_v2/costs_dict_capex.csv
Imported opex from /mnt/data/gsa/pypsa

## Plots

In [4]:
plot_config = {
    'idx_group': IDX_GROUP,
    'idx_group_name': IDX_GROUP_NAME,
    'index_levels_to_drop': INDEX_LEVELS_TO_DROP,
    'set_scen_col_func': set_scen_col,
    'scen_filter': SCEN_FILTER,
    'scen_order': scen_order,
    'fig_kwargs': fig_kwargs,
    'heights': heights,
}

### Electricity Supply

In [5]:
plot_config['chart_config'] = yaml_config['charts']['electricity_supply']

fig, df = plot_energy_balance(
    carrier="AC",
    balance_dict=balance_dict,
    config=plot_config,
    rename_function=rename_electricity,
    show_supply=True,
    save_dir=SDIR
)

fig.show()

### Electricity Demand

In [6]:
plot_config['chart_config'] = yaml_config['charts']['electricity_demand']

fig, df = plot_energy_balance(
    carrier="AC",
    balance_dict=balance_dict,
    config=plot_config,
    rename_function=rename_electricity,
    show_supply=False,
    save_dir=SDIR
)

fig.show()

### Electricity Installed Capacity

In [7]:
plot_config['chart_config'] = yaml_config['charts']['electricity_capacity']

fig, df = plot_capacity(
    carrier="AC",
    capacity_dict=optimal_capacity_dict,
    config=plot_config,
    rename_function=rename_electricity,
    save_dir=SDIR
)

fig.show()

### Hydrogen Balance

In [8]:
plot_config['chart_config'] = yaml_config['charts']['hydrogen_balance']

fig, df = plot_energy_balance(
    carrier="H2",
    balance_dict=balance_dict,
    config=plot_config,
    rename_function=rename_h2,
    save_dir=SDIR
)

fig.show()

### Hydrogen Installed Capacity

In [10]:
plot_config['chart_config'] = yaml_config['charts']['hydrogen_capacity']

fig, df = plot_capacity(
    carrier="H2",
    capacity_dict=optimal_capacity_dict,
    config=plot_config,
    rename_function=rename_h2,
    save_dir=SDIR
)

fig.show()

### Liquid Fuel Balance

In [11]:
plot_config['chart_config'] = yaml_config['charts']['liquid_fuel_balance']

fig, df = plot_energy_balance(
    carrier="oil",
    balance_dict=balance_dict,
    config=plot_config,
    rename_function=rename_oil,
    save_dir=SDIR
)

fig.show()

### CH4 Balance

In [12]:
plot_config['chart_config'] = yaml_config['charts']['ch4_balance']

fig, df = plot_energy_balance(
    carrier="gas",
    balance_dict=balance_dict,
    config=plot_config,
    rename_function=rename_gas,
    save_dir=SDIR
)

fig.show()

### CO2 Capture and Usage

In [13]:
plot_config['chart_config'] = yaml_config['charts']['co2_stored_balance']

fig, df = plot_energy_balance(
    carrier="co2 stored",
    balance_dict=balance_dict,
    config=plot_config,
    rename_function=rename_co2,
    save_dir=SDIR
)

fig.show()

### CO2 Emissions

In [14]:
plot_config['chart_config'] = yaml_config['charts']['co2_emissions']

fig, df = plot_energy_balance(
    carrier="co2",
    balance_dict=balance_dict,
    config=plot_config,
    save_dir=SDIR
)

fig.show()

### System Costs

In [15]:
plot_config['chart_config'] = yaml_config['charts'].get('system_costs', {
    'title': 'System Costs',
    'save_name': 'system_costs_billion_eur',
    'chart_type': 'bar_with_negatives'
})
stats_df = pd.concat([costs_dict["capex"], costs_dict["opex"]], axis=0)
df = prepare_dataframe(stats_df, IDX_GROUP, INDEX_LEVELS_TO_DROP, set_scen_col)
df.variable = df.variable.map(rename_costs)
df = df[df["value"] != 0]
df = df.query("scen in @SCEN_FILTER") if SCEN_FILTER else df

fig_kwargs_costs = plot_config['fig_kwargs'].copy()
fig_kwargs_costs['height'] = plot_config['heights']['large']

fig = px.bar(
    df,
    **fig_kwargs_costs,
    color_discrete_map=colors["costs"],
    category_orders={
        "variable": list(colors["costs"].keys()),
        "scen": scen_order
    },
    labels={
        "value": "System costs [billion EUR/year]",
        "year": "",
        "scen": ""
    }
)

chart_type = plot_config['chart_config'].get('chart_type', 'bar') #'bar_with_negatives'
fig = apply_standard_styling(fig, chart_type, plot_config['chart_config'])

save_name = plot_config['chart_config'].get('save_name', 'system_costs')
save_plotly_fig(df, fig, SDIR, f"{IDX_GROUP_NAME}_{save_name}")

fig.show()